In [1]:
import torch
import numpy as np

## kitti转换世界坐标系脚本

In [20]:
import numpy as np
import os

def read_pose(pose_file):
    poses = []
    with open(pose_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 12:  
                matrix = np.array([float(part) for part in parts] + [0, 0, 0, 1]).reshape(4, 4)
                poses.append(matrix)
    return poses

def transform_points(points, matrix):
    points_hom = np.hstack((points[:, :3], np.ones((points.shape[0], 1))))
    points_world = matrix @ points_hom.T
    return points_world.T[:, :3]

### **转换坐标**

In [24]:
velodyne_dir = "./dataset/sequences/00/velodyne"
pose_file = "./dataset/sequences/00/poses.txt"
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

poses = read_pose(pose_file)

for i, matrix in enumerate(poses):
  
    bin_path = os.path.join(velodyne_dir, f"{i:06d}.bin")
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)[:, :3]

    points_world = transform_points(points, matrix)

    output_path = os.path.join(output_dir, f"{i:06d}.txt")
    np.savetxt(output_path, points_world[:, :3], fmt='%f', delimiter=',')


### **计算最大最小值**

In [30]:
directory = "./output"

global_min = np.array([np.inf, np.inf, np.inf]) 
global_max = np.array([-np.inf, -np.inf, -np.inf]) 


for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        file_path = os.path.join(directory, filename)

        data = np.loadtxt(file_path, delimiter=',')
        
        file_min = np.min(data, axis=0)
        file_max = np.max(data, axis=0)
        
        global_min = np.minimum(global_min, file_min)
        global_max = np.maximum(global_max, file_max)

print("Global min XYZ:", global_min)
print("Global max XYZ:", global_max)

Global min XYZ: [-352.736161  -87.301469  -98.42608 ]
Global max XYZ: [362.363345  84.200387 555.831885]


### **相关函数测试**

In [27]:
mat = read_pose('dataset/sequences/00/poses.txt')
mat

[array([[ 1.00000e+00,  9.31323e-10, -3.27418e-11,  0.00000e+00],
        [-9.31323e-10,  1.00000e+00, -4.65661e-10,  7.45058e-09],
        [ 1.09139e-11, -9.31323e-10,  1.00000e+00,  0.00000e+00],
        [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.00000e+00]]),
 array([[ 0.999991  , -0.00316351, -0.00274942, -0.00135393],
        [ 0.00316045,  0.999994  , -0.00111659, -0.0248245 ],
        [ 0.00275294,  0.00110789,  0.999996  ,  0.672716  ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.999969  , -0.00274641, -0.00740973, -0.00497909],
        [ 0.00272564,  0.999992  , -0.00281192, -0.0381271 ],
        [ 0.0074174 ,  0.00279163,  0.999969  ,  1.37469   ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.999925  , -0.00226129, -0.0120078 , -0.0149304 ],
        [ 0.0022199 ,  0.999992  , -0.00345849, -0.0500735 ],
        [ 0.0120155 ,  0.00343158,  0.999922  ,  2.08592   ],
        [ 0.        ,  0.        ,  0.        , 

In [29]:
points = np.array([[1, 0, 0], [0, 1, 0]], dtype=np.float32)
point_min = np.min(points, axis=0)
point_max = np.max(points, axis=0)
print(point_min, point_max)
transform_points(points, mat[12])

[0. 0. 0.] [1. 1. 0.]


array([[ 0.699613  , -0.17704704,  9.1727435 ],
       [-0.29488158,  0.827369  ,  9.12661205]])

In [18]:
res = np.fromfile("./dataset/sequences/00/velodyne/000000.bin", dtype=np.float32).reshape(-1, 4)[:, :3]
print(res.shape)
with open('000000.txt','w') as fp:
    [fp.write(str(item[0])+','+str(item[1]) +','+ str(item[2]) +'\n') for  item in res]


(124668, 3)


## **kitti visualize pose**

In [9]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
import pytransform3d.transformations as pt
from pytransform3d.transform_manager import TransformManager

%matplotlib notebook


In [10]:
def addPose(num, w_list, ax0, s):
    print("num:", num)
    ax = tm.plot_frames_in("o", s=s, ax = ax0, show_name=False, whitelist=w_list[num-1:num])

posefile = open("./dataset/sequences/00/poses.txt").readlines()

tm = TransformManager()
max_m = [0, 0, 0]
min_m = [0, 0, 0]
w_list = []
for i, l in enumerate(tqdm(posefile[::15])):
    m = np.array(l[:-1].split(' ')).reshape(3,4)
    r = m[:3,:3]
    t = m[:3,3]
    for j in range(3):
        max_m[j] = max(max_m[j], float(t[j]))
        min_m[j] = min(min_m[j], float(t[j]))
    pose = pt.transform_from(r, t)
    tm.add_transform("p"+str(i), "o", pose)
    w_list.append("p"+str(i))

fig = plt.figure(figsize=(8, 8))
fig.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
        hspace = 0, wspace = 0)
# fig.tight_layout()

scale = max([i-j for i, j in zip(max_m, min_m)])/1.5
ax = tm.plot_frames_in("o", s=scale/50, show_name=False, whitelist=w_list[:1])
centor = [(i+j)/2.0 for i, j in zip(max_m, min_m)]

ax.set_xlim((centor[0]-scale, centor[0]+scale))
ax.set_ylim((centor[1]-scale, centor[1]+scale))
ax.set_zlim((centor[2]-scale, centor[2]+scale))

ax.view_init(-0, -90)
# plt.show()
fargs = [w_list, ax, scale/50]
rot_animation = animation.FuncAnimation(fig, addPose, frames=np.arange(0, len(w_list), 1), fargs=fargs, interval=40)
writer = FFMpegWriter(fps=20, codec='h264_nvenc')
rot_animation.save('animation.mp4', writer=writer)
plt.show()

  0%|          | 0/303 [00:00<?, ?it/s]

100%|██████████| 303/303 [00:00<00:00, 550.45it/s] 


<IPython.core.display.Javascript object>

num: 0
num: 0
num: 1
num: 2
num: 3
num: 4
num: 5
num: 6
num: 7
num: 8
num: 9
num: 10
num: 11
num: 12
num: 13
num: 14
num: 15
num: 16
num: 17
num: 18
num: 19
num: 20
num: 21
num: 22
num: 23
num: 24
num: 25
num: 26
num: 27
num: 28
num: 29
num: 30
num: 31
num: 32
num: 33
num: 34
num: 35
num: 36
num: 37
num: 38
num: 39
num: 40
num: 41
num: 42
num: 43
num: 44
num: 45
num: 46
num: 47
num: 48
num: 49
num: 50
num: 51
num: 52
num: 53
num: 54
num: 55
num: 56
num: 57
num: 58
num: 59
num: 60
num: 61
num: 62
num: 63
num: 64
num: 65
num: 66
num: 67
num: 68
num: 69
num: 70
num: 71
num: 72
num: 73
num: 74
num: 75
num: 76
num: 77
num: 78
num: 79
num: 80
num: 81
num: 82
num: 83
num: 84
num: 85
num: 86
num: 87
num: 88
num: 89
num: 90
num: 91
num: 92
num: 93
num: 94
num: 95
num: 96
num: 97
num: 98
num: 99
num: 100
num: 101
num: 102
num: 103
num: 104
num: 105
num: 106
num: 107
num: 108
num: 109
num: 110
num: 111
num: 112
num: 113
num: 114
num: 115
num: 116
num: 117
num: 118
num: 119
num: 120
num: 121
num: 